In [1]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [2]:
def read_data():
    f=open("iris(150).csv", 'r')
    lines=[line.strip() for line in f.readlines()]
    f.close()
    
    lines=[line.split(",") for line in lines if line]
    #print('cool guy', lines)
    lines = lines[1:]
    #print(lines)
    #print([line[:6] for line in lines if line])
    
        
    data=np.array([line[1:5] for line in lines if line], dtype=np.float)
    
    class1=np.array([line[1:5] for line in lines if line[-1]=="setosa"], dtype=np.float)
    
    class2=np.array([line[1:5] for line in lines if line[-1]=="virginica"], dtype=np.float)
    
    class3=np.array([line[1:5] for line in lines if line[-1]=="versicolor"], dtype=np.float)
    
    #list of class labels
    labels=[]
    for line in lines:
        #print(line)
        strt=line.pop()
        #print(strt)
        labels.append(strt)
    
    #print(labels)
    #create array of labels
    labels=[line.split(",") for line in labels if line]
    t=np.zeros(shape=(150, 3))
    #create target vector encoded according to 1-of-K scheme
    for i in range(len(data)):
        if labels[i]==["setosa"]: 
            t[i][0]=1
        elif labels[i]==["versicolor"]: 
            t[i][1]=1
        elif labels[i]==["virginica"]: 
            t[i][2]=1
    
    return class1, class2, class3, data, t


In [3]:
def gaussian(x, mean, cov):
    xm=np.reshape((x-mean), (-1, 1))
    px=1/(math.pow(2.0*math.pi, 2))*1/math.sqrt(np.linalg.det(cov))*math.exp(-(np.dot(np.dot(xm.T, np.linalg.inv(cov)), xm))/2)
    return px

In [39]:
def main():
    class1, class2, class3, data, t=read_data()
    
    
    count=np.zeros(shape=(150,1))
    t_assigned=np.zeros(shape=(150, 3))
    cov=np.zeros(shape=(3, 4, 4))
    mean=np.zeros(shape=(3, 4))
    
    """a = list(zip(class1, class2, class3, data, t))
    random.shuffle(a)
    class1, class2, class3, data, t = zip(*a)
    
    
    class1 = np.asarray(class1)
    class2 = np.asarray(class2)
    class3 = np.asarray(class3)
    
    data = np.asarray(data)
    t = np.asarray(t)"""
    
    class1 = class1[:30]
    class2 = class2[:30]
    class3 = class3[:30]
    
    #print(class1)
    
    
    #compute means for each class
    mean1=class1.mean(axis=0)
    mean2=class2.mean(axis=0)
    mean3=class3.mean(axis=0)
    
        
    
    
    #print(mean1,' ',mean2, ' ', mean3)
    
    #compute covariance matrices, such that the columns are variables and rows are observations of variables
    cov1=np.cov(class1, rowvar=0)
    cov2=np.cov(class2, rowvar=0)
    cov3=np.cov(class3, rowvar=0)
    
    
    
    #print(cov1,' ',cov2, ' ',cov3)
    
    #compute gaussian likelihood functions p(x|Ck) for each class
    for i in range(30, 50):
        px1=(1/3.0)*gaussian(data[i], mean1, cov1)
        px2=(1/3.0)*gaussian(data[i], mean2, cov2)
        px3=(1/3.0)*gaussian(data[i], mean3, cov3)
        m=np.max([px1, px2, px3])
    #compute posterior probability p(Ck|x) assuming that p(x|Ck) is gaussian and the entire expression is wrapped by sigmoid function 
        pc1=((math.exp(px1)*math.exp(-m))*math.exp(m))/((math.exp(px2)*math.exp(-m)+math.exp(px3)*math.exp(-m))*math.exp(m))
        pc2=((math.exp(px2)*math.exp(-m))*math.exp(m))/((math.exp(px1)*math.exp(-m)+math.exp(px3)*math.exp(-m))*math.exp(m))
        pc3=((math.exp(px3)*math.exp(-m))*math.exp(m))/((math.exp(px1)*math.exp(-m)+math.exp(px2)*math.exp(-m))*math.exp(m))
        
        pc2 = np.random.normal(size = [1, 1])[0][0]
        pc3 = np.random.normal(size = [1, 1])[0][0]
        
        
        
        print(pc1,' ',pc2,' ', pc3)
        #assign p(Ck|x)=1 if p(Ck|x)>>p(Cj|x) for all j!=k
        if pc1>pc2 and pc1>pc3: t_assigned[i][0]=1
        elif pc3>pc1 and pc3>pc2: t_assigned[i][1]=1
        elif pc2>pc1 and pc2>pc3: t_assigned[i][2]=1
    #count the number of misclassifications
        for j in range(3):
            if t[i][j]-t_assigned[i][j]!=0: count[i]=1

    cov=[cov1, cov2, cov3]
    mean=[mean1, mean2, mean3]

    t1=np.zeros(shape=(len(class1), 1))
    t2=np.zeros(shape=(len(class2), 1))
    t3=np.zeros(shape=(len(class3), 1))
    for i in range(len(data)):
        for j in range(len(class1)):
            if t_assigned[i][0]==1: t1[j]=1
            elif t_assigned[i][1]==1: t2[j]=2
            elif t_assigned[i][2]==1: t3[j]=3


    print ("number of misclassifications", sum(count) )#, "assigned labels to data points", t_assigned, "target data", t)
    
main()

17.634746606133234   1.00835849475   -1.08403563951
0.551982958188397   -0.0234306445426   0.0483684024837
0.5003642519291877   1.81218503806   0.0817990839016
0.5426705714771964   0.318734313625   -0.42968130277
18.43512387196743   -0.596328815382   -0.320714276742
0.9079412189599328   0.233683914832   0.0816765475137
0.6704872273443723   1.67573983398   -1.47924690625
0.723949928844012   1.86698081131   0.486531048612
2.0354138346141486   1.55561770924   0.208158556976
199.0287603618886   0.156682488872   0.0327162261872
11.095058342256719   1.10732492145   0.943061615156
0.5000010461013221   0.380259308125   0.338642860381
1.5935921410829825   -0.314159590403   -0.796197968351
0.5006242524116264   0.486730582809   0.0162989032123
0.5282877886710837   1.2347490033   -0.672881388483
0.7505826421359563   -0.693627872219   1.26226732394
0.7235656987257162   0.722496099702   1.22645432326
22.467134110623803   -1.81990936804   0.821797209285
10.820197641050617   1.0144372373   1.401305053